# Evaluate likelihood ratio

In [1]:
import sys, os
sys.path.append('../')

import logging
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import colorConverter
from scipy.stats import norm
from sklearn.metrics import roc_curve

from inference.utils import s_from_r, shuffle
import paper_settings

logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.INFO
)

## Setting

In [2]:
setting = "full"
filename = "alices_full_sgd1e2"
samples = list(range(0,6))

## Data

In [3]:
xs = np.load("../data/samples/x_test_{}_point.npy".format(setting))
llrs = np.load("../data/results/llr_{}_grid.npy".format(filename))


## Parameter grids

In [4]:
resolution = 25

f_sub_1d = np.linspace(0.001, 0.200, resolution)
beta_1d = np.linspace(-2.5, -1.5, resolution)

theta0, theta1 = np.meshgrid(f_sub_1d, beta_1d)
theta_grid = np.vstack((theta0.flatten(), theta1.flatten())).T

bin_size = f_sub_1d[1] - f_sub_1d[0]
alpha_edges = np.linspace(f_sub_1d[0] - bin_size/2, f_sub_1d[-1] + bin_size/2, resolution + 1)
bin_size = beta_1d[1] - beta_1d[0]
beta_edges = np.linspace(beta_1d[0] - bin_size/2, beta_1d[-1] + bin_size/2, resolution + 1)

## Plotting options

In [5]:
ncols = 4
nrows = 2

In [6]:
gradmin, gradmax = -0.1, 0.1
gradrelmax = 0.002
xmin, xmax = 2.3, 3.2

llrmin, llrmax = 0., 20.
llr_cmap = "viridis_r"
normalize_to_mle = True


In [7]:
fig, gs = paper_settings.grid2(ncols, nrows, height=3.*nrows)

for i in range(ncols):
    ax = plt.subplot(gs[i])
    im = plt.imshow(
        np.log10(xs[i_img]),
        vmin=xmin,
        vmax=xmax,
        cmap='gist_gray',
        extent=(-3.2,3.2,-3.2,3.2),
        origin="lower",
        alpha=1.
    )
    if i == 0:
        plt.plot([-2.9, -1.9], [-2.9, -2.9], c="white", lw=1.5, ls="-")
        plt.text(-2.4, -2.65, '$1$"', va="center", ha="center", color="white")
        
    plt.xlim(-3.2,3.2)
    plt.ylim(-3.2,3.2)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
        
    if i == 0:
        ax = plt.subplot(gs[ncols])
        cbar = plt.colorbar(im, cax=ax)
        cbar.set_label(r'Intensity')
        
        
    
    ax = plt.subplot(gs[ncols + 1 + i])
    z = -2. * llrs[k][:, i_img]
    z = z - np.min(z)
    z = np.clip(z, llrmin, llrmax)
    pcm = ax.pcolormesh(
        alpha_edges,
        beta_edges,
        z.reshape((resolution, resolution)),
        norm=matplotlib.colors.Normalize(vmin=llrmin, vmax=llrmax),
        cmap=llr_cmap
    )
    cs = plt.contour(
        0.5 * (alpha_edges[1:] + alpha_edges[:-1]),
        0.5 * (beta_edges[1:] + beta_edges[:-1]),
        z.reshape((resolution, resolution)),
        [5.991464547107983],
        colors="black",
        linewidths=1.,
        linestyles=["-"]
    )
    plt.scatter(0.05, -1.9, s=60., color='black', marker='*')
    
    plt.xlabel(r'$f_{\mathrm{sub}}$')
    if i == 0:
        plt.ylabel(r'$\beta$')
    
    if i == 0:
        cbar = fig.colorbar(pcm, ax=ax, extend='both')
        cbar.set_label(r'$-2\; \log \;\hat{r}(x | \beta, f_{\mathrm{sub}})$')
        plt.xlim(alpha_edges[0], alpha_edges[-1])
        plt.ylim(beta_edges[0], beta_edges[-1])

    
plt.savefig("../figures/individual_lens_predictions.pdf")
    

ValueError: Expected the given number of height ratios to match the number of rows of the grid

<Figure size 881.28x432 with 0 Axes>